# Задание 1.
Обязательная часть
Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

### Дополнительная часть (необязательная)
Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст_статьи>

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [ ]:
KEYWORDS = ['python', 'парсинг', 'разработчик', 'ученые']

res =  requests.get('https://habr.com/ru/all/')
soup = BeautifulSoup(res.text, 'html.parser') 
links = soup.find_all('h2', class_='post__title')
# извлечение всех тегов для ссылок
link_list = list(map(lambda x: x.find('a').get('href'), links))
# извлечение ссылок
link_list

In [ ]:
def get_habr_news(link_list):
    habr_news = pd.DataFrame()
    for link in link_list:
        soup = BeautifulSoup(requests.get(link).text, 'html.parser')
#         запрос к ссылке
        time.sleep(0.3)
#     задержка на всякий случай
        text = soup.find('div', class_ = 'post__body post__body_full').text.lower().strip()
#     получение текста поста
        for words in KEYWORDS:
            if words in text:
#                 поиск нужных слов
                date = pd.to_datetime(soup.find('span', class_='post__time').get('data-time_published'), dayfirst = True).date()
#     получение даты
                title = soup.find('span', class_ = 'post__title-text').text
#     получение загловка
                row = {'date': date, 'title': title, 'link': link, 'text': text}
                habr_news = pd.concat([habr_news, pd.DataFrame([row])])  
#         склеивание в dataframe
    return habr_news
get_habr_news(link_list)

# Задание 2.
Обязательная часть:

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <дата утечки> - <источник утечки> - <описание утечки>

Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

In [ ]:
EMAIL = ['xxx@x.ru', 'yyy@y.com']
emails = {
    'emailAddresses': 'xxx@x.ru',
    'emailAddresses': 'yyy@y.com'
}
url = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
headers = {
    'Vaar-Version': '0',
    'Vaar-Header-App-Product': 'hackcheck-web-avast', 
    'Vaar-Header-App-Build-Version': '1.0.0',
    'Vaar-Header-App-Product-Name': 'hackcheck-web-avast'
}

In [ ]:
emails_df = pd.DataFrame()
for item in EMAIL:
    r = requests.post(url, json = {'emailAddresses': [item]}, headers = headers)
    time.sleep(0.35)
    for email in r:
        emails_df = pd.concat([emails_df, pd.DataFrame(r.json()['breaches']).transpose()])
        emails_df['publishDate'] = pd.to_datetime(emails_df['publishDate'])
        emails_df.loc[:, ['publishDate', 'site', 'description']]
    emails_df